<a href="https://colab.research.google.com/github/mannb986/admissions_data_deep_learning/blob/main/admissions_data_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Regression with Admissions Data

The goal of this project is to create a regression model that predicts the likelihood that a student applying to graduate school will be accpeted based on various application factors (such as test scores). 

By analysing the parameters in the graduate admissions dataset I will use TensorFlow & Keras to create a regression model that can evaluate the chances of an applicant being admitted. 

## Data Loading & Observing

In [1]:
 from google.colab import files


uploaded = files.upload()

Saving admissions_data.csv to admissions_data (1).csv


In [2]:
import pandas as pd

admissions_df = pd.read_csv('admissions_data.csv')

In [3]:
admissions_df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [4]:
admissions_df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


The above shows the various parameters the admission officers use to evaluate university applicants. There is information on 500 applicants from various universities. 

All columns are numerical so won't requires any one-hot encoding. 

My features for the model will include all columns except `Serial No.` and `Chance of Admit`. `Chance of Admit` will be my labels for the model. 

In [5]:
admissions_df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [6]:
features = admissions_df.iloc[:, 1:-1]

In [7]:
features.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0


In [8]:
labels = admissions_df.iloc[:, -1]

In [9]:
labels.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64

## Data Processing

In [10]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.33, random_state = 42)

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 

numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns

ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

In [12]:
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)

In [13]:
features_train_scaled = pd.DataFrame(features_train_scaled, columns = features_train.columns)
features_test_scaled = pd.DataFrame(features_test_scaled, columns = features_test.columns)

In [14]:
features_test_scaled.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,1.570357,1.423757,0.784816,0.621518,-0.001613,1.612511,0.873589
1,-0.251845,0.112531,0.784816,1.123225,0.538819,0.768277,0.873589
2,-0.160734,-0.379179,-0.950586,-1.385311,-1.082478,-1.578691,-1.144703
3,-0.434065,0.276434,-0.082885,-0.381897,-0.542045,0.177314,-1.144703
4,0.841476,0.768144,-0.082885,0.119811,-0.542045,0.785162,0.873589


## Building the Model

In [15]:
from tensorflow.keras.models import Sequential

my_model = Sequential()

In [16]:
from tensorflow.keras.layers import InputLayer

#creating the input layer with the shape corresponding to the number of features
input = InputLayer(input_shape=(features.shape[1],))
my_model.add(input)

In [17]:
from tensorflow.keras.layers import Dense

#adding one hidden layer with 128 neurons
my_model.add(Dense(128, activation='relu'))

In [18]:
#adding output layer with a single output
my_model.add(Dense(1))

In [19]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


## Initializing the Optimizer & Compiling the Model

In [20]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.01)

my_model.compile(loss = 'mse', metrics = 'mae', optimizer=opt)

## Fit & Evaluate Model

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=40) 

history = my_model.fit(features_train_scaled, labels_train, epochs=40, batch_size=4, verbose=1, validation_split=0.2, callbacks=[stop])


Epoch 1/40
67/67 [==============================] - 1s 10ms/step - loss: 0.0985 - mae: 0.2378 - val_loss: 0.0216 - val_mae: 0.1255
Epoch 2/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0176 - mae: 0.0998 - val_loss: 0.0121 - val_mae: 0.0903
Epoch 3/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0121 - mae: 0.0914 - val_loss: 0.0168 - val_mae: 0.1039
Epoch 4/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0131 - mae: 0.0852 - val_loss: 0.0085 - val_mae: 0.0755
Epoch 5/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0102 - mae: 0.0782 - val_loss: 0.0047 - val_mae: 0.0558
Epoch 6/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0049 - mae: 0.0571 - val_loss: 0.0053 - val_mae: 0.0591
Epoch 7/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0041 - mae: 0.0508 - val_loss: 0.0123 - val_mae: 0.0765
Epoch 8/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0074 - mae: 0.0

In [22]:
res_mse, rse_mae = my_model.evaluate(features_test_scaled, labels_test, verbose=0)

In [23]:
print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  0.004572281613945961
MAE  0.049801815301179886


Visualise the model's training progress using the stats stored in the history object

In [26]:
hist = pd.DataFrame(history.history)
hist['EPOCH'] = history.epoch
hist.tail()

,loss,mae,val_loss,val_mae,EPOCH
35,0.004517,0.051951,0.005470,0.063838,35
36,0.003872,0.046571,0.003963,0.047711,36
37,0.004051,0.046263,0.003812,0.049658,37
38,0.004565,0.051057,0.004280,0.046507,38
39,0.003659,0.045621,0.003617,0.046171,39


The results show a very good performance of the model. It shows the model has a MAE of around 0.05 which equates to being around 5% off when predicting the students chance of admission. 

I will tune the model to understand the effects of tuning the hyperparameters. 

## Tuning Model

In [27]:
##Changing batch size from 1 to 4
history = my_model.fit(features_train_scaled, labels_train, epochs=40, batch_size=4, verbose=1, validation_split=0.2, callbacks=[stop])

Epoch 1/40
67/67 [==============================] - 0s 4ms/step - loss: 0.0037 - mae: 0.0448 - val_loss: 0.0030 - val_mae: 0.0395
Epoch 2/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0033 - mae: 0.0424 - val_loss: 0.0038 - val_mae: 0.0435
Epoch 3/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0040 - mae: 0.0474 - val_loss: 0.0041 - val_mae: 0.0490
Epoch 4/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0037 - mae: 0.0457 - val_loss: 0.0057 - val_mae: 0.0539
Epoch 5/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0048 - mae: 0.0527 - val_loss: 0.0037 - val_mae: 0.0436
Epoch 6/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0045 - mae: 0.0524 - val_loss: 0.0042 - val_mae: 0.0476
Epoch 7/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0035 - mae: 0.0439 - val_loss: 0.0039 - val_mae: 0.0443
Epoch 8/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.04

In [28]:
res_mse, rse_mae = my_model.evaluate(features_test_scaled, labels_test, verbose=0)

print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  0.005015557166188955
MAE  0.05474288761615753


Comparing the results with where the batch size was 1, it can seen there is very little difference in the results. 

I will add another hiddent layer to the model. 

In [29]:
##adding another hidden layer to the model.
my_model_2 = Sequential()
input_2 = InputLayer(input_shape=(features.shape[1],))
my_model_2.add(input_2)
my_model_2.add(Dense(128, activation='relu'))
my_model_2.add(Dense(64, activation='relu'))
my_model_2.add(Dense(1))
my_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 9,345
Trainable params: 9,345
Non-trainable params: 0
_________________________________________________________________


In [30]:
my_model_2.compile(loss = 'mse', metrics = 'mae', optimizer=opt)

In [31]:
my_model_2.fit(features_train_scaled, labels_train, epochs=40, batch_size=4, verbose=1, validation_split=0.2, callbacks=[stop])

Epoch 1/40
67/67 [==============================] - 1s 6ms/step - loss: 0.7618 - mae: 0.6328 - val_loss: 0.0087 - val_mae: 0.0731
Epoch 2/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0077 - mae: 0.0718 - val_loss: 0.0061 - val_mae: 0.0657
Epoch 3/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0053 - mae: 0.0554 - val_loss: 0.0036 - val_mae: 0.0429
Epoch 4/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0047 - mae: 0.0481 - val_loss: 0.0036 - val_mae: 0.0472
Epoch 5/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0052 - mae: 0.0536 - val_loss: 0.0031 - val_mae: 0.0403
Epoch 6/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0048 - mae: 0.0520 - val_loss: 0.0034 - val_mae: 0.0447
Epoch 7/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0035 - mae: 0.0441 - val_loss: 0.0034 - val_mae: 0.0432
Epoch 8/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0044 - mae: 0.04

In [32]:
res_mse, rse_mae = my_model_2.evaluate(features_test_scaled, labels_test, verbose=0)

print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  0.005745344795286655
MAE  0.05882490426301956


By adding a leyer there has been a slight decrease in performance. 